In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [73]:
train_orig = pd.read_csv("/content/train.csv")
test_orig = pd.read_csv("/content/test.csv")

In [74]:
train_orig.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [75]:
test_orig.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [76]:
train_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
 10  pet_category    18834 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 1.6+ MB


In [77]:
train = train_orig.copy()
test = test_orig.copy()

In [78]:
train['issue_year'] = 0
train['listing_year'] = 0

for idx, date in enumerate(train.issue_date):
  train.loc[idx,'issue_year'] = int(date[:4])

for idx, date in enumerate(train.listing_date):
  train.loc[idx,'listing_year'] = int(date[:4])


train['issue_month'] = 0
train['listing_month'] = 0

for idx, date in enumerate(train.issue_date):
  train.loc[idx,'issue_month'] = int(date[5:7])

for idx, date in enumerate(train.listing_date):
  train.loc[idx,'listing_month'] = int(date[5:7])


train['issue_day'] = 0
train['listing_day'] = 0

for idx, date in enumerate(train.issue_date):
  train.loc[idx,'issue_day'] = int(date[8:10])

for idx, date in enumerate(train.listing_date):
  train.loc[idx,'listing_day'] = int(date[8:10])


train.drop(['issue_date','listing_date'],axis=1,inplace=True)

train.head()

,pet_id,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category,issue_year,listing_year,issue_month,listing_month,issue_day,listing_day
0,ANSL_69903,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1,2016,2016,7,9,10,21
1,ANSL_66892,1.0,White,0.72,14.19,13,9,0.0,2,2013,2018,11,12,21,27
2,ANSL_69750,NaN,Brown,0.15,40.90,15,4,2.0,4,2014,2016,9,10,28,19
3,ANSL_71623,1.0,White,0.62,17.82,0,1,0.0,2,2016,2019,12,1,31,25
4,ANSL_57969,2.0,Black,0.50,11.06,18,4,0.0,1,2017,2017,9,11,28,19


In [79]:
test['issue_year'] = 0
test['listing_year'] = 0

for idx, date in enumerate(test.issue_date):
  test.loc[idx,'issue_year'] = int(date[:4])

for idx, date in enumerate(test.listing_date):
  test.loc[idx,'listing_year'] = int(date[:4])


test['issue_month'] = 0
test['listing_month'] = 0

for idx, date in enumerate(test.issue_date):
  test.loc[idx,'issue_month'] = int(date[5:7])

for idx, date in enumerate(test.listing_date):
  test.loc[idx,'listing_month'] = int(date[5:7])


test['issue_day'] = 0
test['listing_day'] = 0

for idx, date in enumerate(test.issue_date):
  test.loc[idx,'issue_day'] = int(date[8:10])

for idx, date in enumerate(test.listing_date):
  test.loc[idx,'listing_day'] = int(date[8:10])


test.drop(['issue_date','listing_date'],axis=1,inplace=True)

test.head()

,pet_id,condition,color_type,length(m),height(cm),X1,X2,issue_year,listing_year,issue_month,listing_month,issue_day,listing_day
0,ANSL_75005,0.0,Black,0.87,42.73,0,7,2005,2017,8,9,17,7
1,ANSL_76663,1.0,Orange Tabby,0.06,6.71,0,1,2018,2019,11,5,15,8
2,ANSL_58259,1.0,Black,0.24,41.21,0,7,2012,2018,10,4,11,2
3,ANSL_67171,1.0,Black,0.29,8.46,7,1,2015,2018,2,4,13,6
4,ANSL_72871,1.0,Brown,0.71,30.92,0,7,2017,2018,1,4,18,26


In [82]:
"""color_map = {'Brown Tabby':'Black-Brown','Brown Brindle':'Black-Brown','Brown Merle':'Black-Brown','Brown Tiger':'Black-Brown','Brown':'Black-Brown',
             'Chocolate':'Black-Brown','Chocolate Point':'Black-Brown','Liver':'Black-Brown','Liver Tick':'Black-Brown',
             'Torbie':'Black-Brown','Sable':'Black-Brown','Buff':'Black-Brown','Seal Point':'Black-Brown',
             'Tortie Point':'Tortie','Tortie':'Tortie','Agouti':'Black-Brown',
             'Black Brindle':'Black-Brown','Black Tiger':'Black-Brown','Black Tabby':'Black-Brown','Black Smoke':'Black-Brown','Black':'Black-Brown',

             'Red':'Reddish-Orange','Red Tick':'Reddish-Orange','Red Merle':'Reddish-Orange','Apricot':'Reddish-Orange','Orange':'Reddish-Orange',
             'Yellow':'Yellowish-Orange','Tan':'Yellowish-Orange','Gold':'Yellowish-Orange','Yellow Brindle':'Yellowish-Orange','Orange Tabby':'Reddish-Orange',

             'Gray':'Gray','Gray Tabby':'Gray','Calico':'Gray','Calico Point':'Gray',
             'Silver':'Silver','Silver Tabby':'Silver','Silver Lynx Point':'Silver',

             'Cream':'Cream','Cream Tabby':'Cream','Fawn':'Cream','Lynx Point':'Cream','White':'Cream','Flame Point':'Cream','Lilac Point':'Cream',
             
             'Tricolor':'Other','Pink':'Other','Green':'Other',

             'Blue Tabby':'Blue','Blue Merle':'Blue','Blue Tick':'Blue','Blue Point':'Blue','Blue Tiger':'Blue','Blue Cream':'Blue','Blue Smoke':'Blue','Blue':'Blue',
             }

train.color_type = train.color_type.map(color_map, na_action = 'ignore')
test.color_type = test.color_type.map(color_map, na_action = 'ignore')"""

"color_map = {'Brown Tabby':'Black-Brown','Brown Brindle':'Black-Brown','Brown Merle':'Black-Brown','Brown Tiger':'Black-Brown','Brown':'Black-Brown',\n             'Chocolate':'Black-Brown','Chocolate Point':'Black-Brown','Liver':'Black-Brown','Liver Tick':'Black-Brown',\n             'Torbie':'Black-Brown','Sable':'Black-Brown','Buff':'Black-Brown','Seal Point':'Black-Brown',\n             'Tortie Point':'Tortie','Tortie':'Tortie','Agouti':'Black-Brown',\n             'Black Brindle':'Black-Brown','Black Tiger':'Black-Brown','Black Tabby':'Black-Brown','Black Smoke':'Black-Brown','Black':'Black-Brown',\n\n             'Red':'Reddish-Orange','Red Tick':'Reddish-Orange','Red Merle':'Reddish-Orange','Apricot':'Reddish-Orange','Orange':'Reddish-Orange',\n             'Yellow':'Yellowish-Orange','Tan':'Yellowish-Orange','Gold':'Yellowish-Orange','Yellow Brindle':'Yellowish-Orange','Orange Tabby':'Reddish-Orange',\n\n             'Gray':'Gray','Gray Tabby':'Gray','Calico':'Gray','Calico Po

In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   condition       17357 non-null  float64
 2   color_type      18834 non-null  object 
 3   length(m)       18834 non-null  float64
 4   height(cm)      18834 non-null  float64
 5   X1              18834 non-null  int64  
 6   X2              18834 non-null  int64  
 7   breed_category  18834 non-null  float64
 8   pet_category    18834 non-null  int64  
 9   issue_year      18834 non-null  int64  
 10  listing_year    18834 non-null  int64  
 11  issue_month     18834 non-null  int64  
 12  listing_month   18834 non-null  int64  
 13  issue_day       18834 non-null  int64  
 14  listing_day     18834 non-null  int64  
dtypes: float64(4), int64(9), object(2)
memory usage: 2.2+ MB


In [84]:
train.set_index('pet_id',inplace=True)
test.set_index('pet_id',inplace=True)

In [85]:
train.condition.fillna(3,inplace=True)
test.condition.fillna(3,inplace=True)

In [86]:
train["diff_issue_list"] = train["listing_year"] - train["issue_year"]
test["diff_issue_list"] = test["listing_year"] - test["issue_year"]

#train["duration_at_shelter"] = 2020 - train["listing_year"]
#test["duration_at_shelter"] = 2020 - test["listing_year"]

#train["issue_year"] = train["issue_year"]/2020.
#test["issue_year"] = test["issue_year"]/2020.

#train.drop(["issue_year","listing_year"],axis=1,inplace=True)
#test.drop(["issue_year","listing_year"],axis=1,inplace=True)

In [87]:
Y1 = train['breed_category']
Y2 = train['pet_category']
train.drop(['breed_category','pet_category'],axis=1,inplace=True)

In [88]:
train

,condition,color_type,length(m),height(cm),X1,X2,issue_year,listing_year,issue_month,listing_month,issue_day,listing_day,diff_issue_list
pet_id,,,,,,,,,,,,,
ANSL_69903,2.0,Brown Tabby,0.80,7.78,13,9,2016,2016,7,9,10,21,0
ANSL_66892,1.0,White,0.72,14.19,13,9,2013,2018,11,12,21,27,5
ANSL_69750,3.0,Brown,0.15,40.90,15,4,2014,2016,9,10,28,19,2
ANSL_71623,1.0,White,0.62,17.82,0,1,2016,2019,12,1,31,25,3
ANSL_57969,2.0,Black,0.50,11.06,18,4,2017,2017,9,11,28,19,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ANSL_51738,2.0,Tricolor,0.44,27.36,0,1,2017,2018,1,3,26,9,1
ANSL_59900,3.0,Brown,0.73,14.25,15,4,2016,2017,6,7,18,9,1
ANSL_53210,0.0,Calico Point,0.99,28.13,13,9,2010,2018,7,8,21,22,8


In [89]:
test

,condition,color_type,length(m),height(cm),X1,X2,issue_year,listing_year,issue_month,listing_month,issue_day,listing_day,diff_issue_list
pet_id,,,,,,,,,,,,,
ANSL_75005,0.0,Black,0.87,42.73,0,7,2005,2017,8,9,17,7,12
ANSL_76663,1.0,Orange Tabby,0.06,6.71,0,1,2018,2019,11,5,15,8,1
ANSL_58259,1.0,Black,0.24,41.21,0,7,2012,2018,10,4,11,2,6
ANSL_67171,1.0,Black,0.29,8.46,7,1,2015,2018,2,4,13,6,3
ANSL_72871,1.0,Brown,0.71,30.92,0,7,2017,2018,1,4,18,26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ANSL_66809,2.0,Brown,0.82,36.08,13,9,2016,2017,2,3,10,10,1
ANSL_59041,0.0,Tan,0.49,27.54,13,9,2015,2018,12,2,7,12,3
ANSL_60034,0.0,Black,0.98,37.19,0,7,2015,2017,12,1,8,4,2


In [114]:
from sklearn.model_selection import train_test_split

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(train, Y1, train_size=0.9, test_size=0.1, random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = test[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [115]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error

model = XGBClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_valid)
mae = mean_absolute_error(predictions, y_valid)
print("MAE of XGB:" , mae)

model = LGBMClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_valid)
mae = mean_absolute_error(predictions, y_valid)
print("MAE of LGBM:" , mae)

MAE of XGB: 0.09182590233545647
MAE of LGBM: 0.09554140127388536


In [105]:
n_estimators = [100,200,500,1000,1200,1500,2000]
learning_rate = [0.001,0.002,0.005,0.01,0.02,0.05,0.1]

# Obtain all possible combinations
grid_search = [ (lr, n) for lr in learning_rate for n in n_estimators]

for lr, n in grid_search:
  model = XGBClassifier(n_estimators=n,learning_rate=lr)
  model.fit(X_train, y_train)
  predictions = model.predict(X_valid)
  mae = mean_absolute_error(predictions, y_valid)
  print("MAE of XGB with lr=%f and n=%f: %f" % (lr,n,mae))

MAE of XGB with lr=0.001000 and n=100.000000: 0.103503
MAE of XGB with lr=0.001000 and n=200.000000: 0.103503
MAE of XGB with lr=0.001000 and n=500.000000: 0.103503
MAE of XGB with lr=0.001000 and n=1000.000000: 0.103503
MAE of XGB with lr=0.001000 and n=1200.000000: 0.103503
MAE of XGB with lr=0.001000 and n=1500.000000: 0.103503
MAE of XGB with lr=0.001000 and n=2000.000000: 0.103503
MAE of XGB with lr=0.002000 and n=100.000000: 0.103503
MAE of XGB with lr=0.002000 and n=200.000000: 0.103503
MAE of XGB with lr=0.002000 and n=500.000000: 0.103503
MAE of XGB with lr=0.002000 and n=1000.000000: 0.103503
MAE of XGB with lr=0.002000 and n=1200.000000: 0.098726
MAE of XGB with lr=0.002000 and n=1500.000000: 0.096072
MAE of XGB with lr=0.002000 and n=2000.000000: 0.092887
MAE of XGB with lr=0.005000 and n=100.000000: 0.103503
MAE of XGB with lr=0.005000 and n=200.000000: 0.103503
MAE of XGB with lr=0.005000 and n=500.000000: 0.096072
MAE of XGB with lr=0.005000 and n=1000.000000: 0.092887
M

In [116]:
model = XGBClassifier()
model.fit(X_train, y_train)
predictions_1 = model.predict(X_test)

In [118]:
from sklearn.model_selection import train_test_split

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(train, Y2, train_size=0.9, test_size=0.1, random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = test[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [119]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error

model = XGBClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_valid)
mae = mean_absolute_error(predictions, y_valid)
print("MAE of XGB:" , mae)

model = LGBMClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_valid)
mae = mean_absolute_error(predictions, y_valid)
print("MAE of LGBM:" , mae)

MAE of XGB: 0.12632696390658174
MAE of LGBM: 0.11040339702760085


In [109]:
n_estimators = [100,200,500,1000,1200,1500,2000]
learning_rate = [0.005,0.01,0.02,0.05,0.1,0.5]

# Obtain all possible combinations
grid_search = [ (lr, n) for lr in learning_rate for n in n_estimators]

for lr, n in grid_search:
  model = LGBMClassifier(n_estimators=n,learning_rate=lr)
  model.fit(X_train, y_train)
  predictions = model.predict(X_valid)
  mae = mean_absolute_error(predictions, y_valid)
  print("MAE of LGBM with lr=%f and n=%f: %f" % (lr,n,mae))

MAE of LGBM with lr=0.005000 and n=100.000000: 0.224522
MAE of LGBM with lr=0.005000 and n=200.000000: 0.140658
MAE of LGBM with lr=0.005000 and n=500.000000: 0.111996
MAE of LGBM with lr=0.005000 and n=1000.000000: 0.113588
MAE of LGBM with lr=0.005000 and n=1200.000000: 0.114119
MAE of LGBM with lr=0.005000 and n=1500.000000: 0.115180
MAE of LGBM with lr=0.005000 and n=2000.000000: 0.112527
MAE of LGBM with lr=0.010000 and n=100.000000: 0.140658
MAE of LGBM with lr=0.010000 and n=200.000000: 0.113588
MAE of LGBM with lr=0.010000 and n=500.000000: 0.114119
MAE of LGBM with lr=0.010000 and n=1000.000000: 0.114650
MAE of LGBM with lr=0.010000 and n=1200.000000: 0.113057
MAE of LGBM with lr=0.010000 and n=1500.000000: 0.112527
MAE of LGBM with lr=0.010000 and n=2000.000000: 0.110934
MAE of LGBM with lr=0.020000 and n=100.000000: 0.114119
MAE of LGBM with lr=0.020000 and n=200.000000: 0.108811
MAE of LGBM with lr=0.020000 and n=500.000000: 0.113588
MAE of LGBM with lr=0.020000 and n=1000.

In [120]:
model = LGBMClassifier()
model.fit(X_train, y_train)
predictions_2 = model.predict(X_test)

In [121]:
# Save test predictions to file
output = pd.DataFrame({'pet_id': X_test.index,
                       'breed_category': predictions_1,
                       'pet_category': predictions_2})
output.to_csv('submission.csv', index=False)